# Préparation des variables pour les variables de style

In [ ]:
library(tidyverse)
library(factoextra)

# Charger les données
df_pilot1_2022 <- readRDS("../../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/01_pilot1_2022.rds")

In [2]:
# Préparation du jeu de données pour les variables de style
variables_style <- c(
  "id",
  "app_swag_Formel",
  "app_swag_Classique",
  "app_swag_Casual",
  "app_swag_Sport",
  "app_swag_Chic",
  "app_swag_HippBoheme",
  "app_swag_Punk",
  "app_swag_Rock",
  "app_swag_Other",
  "app_noTattoo",
  "animal_cat",
  "animal_dog",
  "animal_domestic",
  "animal_farm",
  "animal_noPet"
)

df_style <- df_pilot1_2022 %>%
  select(all_of(variables_style))

In [ ]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_style) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_style[[v]]))
    cat("\n\n")
  }
}

## Transformations

- app_swag

Les variables app_swag sont telles que certaines catégories sont très peu peuplées. Regrouper les styles pour en faire des styles plus généraux.

- noTatoo

Retourner le sens logique pour que vrai soit avec tatou => deviendra Tatoo

- animal

Regrouper certaines classes d'animal pour avoir des catégories plus générales


In [4]:
# Regroupement de app_swag
#-------------------------

df_style <- df_style %>%

  # Regrouper Formel et Chic sous Classique
  mutate(
    app_swag_Classique = if_else(
      app_swag_Formel == 1 | app_swag_Chic == 1,
      1,
      app_swag_Classique
    )
  ) %>%
  select(-app_swag_Formel, -app_swag_Chic) %>%

  # Regrouper HippBoheme, Punk, Rock sous Other
  mutate(
    app_swag_Other = if_else(
      app_swag_HippBoheme == 1 | app_swag_Punk == 1 | app_swag_Rock == 1,
      1,
      app_swag_Other
    )
  ) %>%
  select(-app_swag_HippBoheme, -app_swag_Punk, -app_swag_Rock)

In [5]:
# Changement de sens logique de NoTatoo
#--------------------------------------

df_style <- df_style %>%
  mutate(app_withTattoo = if_else(app_noTattoo == 1, 0, 1)) %>%
  select(-app_noTattoo)

In [6]:
# Regroupement de animal
#-----------------------

df_style <- df_style %>%
  mutate(animal_other = if_else(
    animal_domestic == 1 | animal_farm == 1,
    1,
    0
  )
  ) %>%
  select(-animal_domestic, -animal_farm)

In [7]:
variables_style_clust <- c(
  "id",
  "app_swag_Classique",
  "app_swag_Casual",
  "app_swag_Sport",
  "app_swag_Other",
  "app_withTattoo",
  "animal_cat",
  "animal_dog",
  "animal_other",
  "animal_noPet"
)

In [ ]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_style_clust) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_style[[v]]))
    cat("\n\n")
  }
}

In [9]:
# Sauvegarder les données préparées
saveRDS(df_style, file = "../../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/02_pilot1_2022_style.rds")

---

### Activité de Clustering

Effectuer un exercice de clustering pour mieux saisir les données et leur relation. 

Cet exercice est purement exploratoire et sert à voir si les données peuvent être utilisées pour faire du clustering.

In [2]:
df_style <- readRDS(file = "../../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/02_pilot1_2022_style.rds")

In [ ]:
library(dplyr)

# Exploration des données
df_clust <- df_style %>% select(-id)

# Aperçu et exploration initiale :

# Aperçu des données
head(df_clust)

# # Vérification des données manquantes
sum(is.na(df_clust))

# # Statistiques descriptives
summary(df_clust)



In [ ]:
# Analyser la distribution des variables pour décider quelle méthode utiliser

# Calcul de la variance pour chaque colonne
variances <- apply(df_clust, 2, var)

# Calcul de l'écart-type pour chaque colonne
std_devs <- apply(df_clust, 2, sd)

# Afficher les résultats
print(variances)
print(std_devs)

# Calcul du ratio max/min
ratio_var <- max(variances) / min(variances)

# Afficher le ratio
print(ratio_var)

# Test de Bartlett sur les colonnes
library(car)
bartlett.test(df_clust)

### Clustering avec kmeans

In [ ]:
library(factoextra)

## Clustering avec k-means
df_scaled <- scale(df_clust)

# Utilisons la méthode de l’élbow (coude) et l’indice de silhouette.

# Méthode du coude
fviz_nbclust(df_scaled, kmeans, method = "wss") +
  ggtitle("Méthode du coude")

# Indice de silhouette
fviz_nbclust(df_scaled, kmeans, method = "silhouette") +
  ggtitle("Indice de silhouette")

In [11]:
# Application de K-Means Clustering

# Choisissez un nombre de clusters approprié (par exemple, 3).
k <- 4

# K-Means Clustering
set.seed(123)  # Pour la reproductibilité
kmeans_result <- kmeans(df_scaled, centers = k, nstart = 25)

# Ajouter les clusters au dataframe original
df_style$cluster_kmeans <- as.factor(kmeans_result$cluster)

In [ ]:
kmeans_result

In [ ]:
# Visualisation des clusters

# Visualisation des clusters dans l’espace des deux premières dimensions :

# Utilisation de factoextra pour représenter les clusters
fviz_cluster(kmeans_result, data = df_scaled, ellipse.type = "euclid") +
  ggtitle("Visualisation des clusters avec K-Means")

# Graphique par variables

# Variables binaires
g_var_bin <- function(df, nom_var) {
  # Distribution des clusters par var
  g <- ggplot(df, aes(x = .data[[nom_var]], fill = cluster_kmeans)) +
    geom_bar(position = "dodge") +
    ggtitle(paste("Distribution des clusters pour", nom_var)) +
    theme_minimal()
  return(g)
}
g_var_bin(df_style, "app_swag_Classique")
g_var_bin(df_style, "app_swag_Casual")
g_var_bin(df_style, "app_swag_Sport")
g_var_bin(df_style, "app_swag_Other")
g_var_bin(df_style, "app_withTattoo")
g_var_bin(df_style, "animal_cat")
g_var_bin(df_style, "animal_dog")
g_var_bin(df_style, "animal_other")
g_var_bin(df_style, "animal_noPet")

### Clustering avec kmodes

In [ ]:
library(cluster)
library(klaR)

# Créer un échantillon de données catégorielles (exemple)
set.seed(123)

# Appliquer KModes pour différents nombres de clusters
n <- 10
wss <- numeric(n)  # Stocker l'inertie pour différents k

# Calculer la silhouette pour différents nombres de clusters
sil_width <- numeric(n)

for (k in 2:n) {
  kmodes_result <- kmodes(df_clust, modes = k, iter.max = 100, weighted = FALSE)
  wss[k] <- sum(kmodes_result$withindiff)  # Somme des distances intra-cluster
  silhouette_result <- silhouette(kmodes_result$cluster, dist(df_clust))  # Utiliser la distance catégorielle
  sil_width[k] <- mean(silhouette_result[, 3])
}

# Tracer le graphique du coude
plot(2:n, wss[2:n], type = "b", pch = 19, col = "blue", xlab = "Nombre de clusters", ylab = "Somme des distances intra-cluster", main = "Méthode du coude")

# Tracer le graphique des scores de silhouette
plot(2:n, sil_width[2:n], type = "b", pch = 19, col = "red", xlab = "Nombre de clusters", ylab = "Score de silhouette", main = "Méthode de silhouette")

In [ ]:
library(klaR)

# Choisissez un nombre de clusters approprié (par exemple, 3).
k <- 7

# Exemple de clustering avec K-Modes
set.seed(123)
kmodes_result <- kmodes(df_clust, modes = k, iter.max = 10, weighted = FALSE)

print(kmodes_result)

# Résultats
kmodes_result$cluster    # Les clusters attribués
kmodes_result$modes      # Les centres des clusters

In [140]:
# Ajouter les clusters au dataframe original
df_style$cluster_kmodes <- as.factor(kmodes_result$cluster)

In [ ]:
# Visualisation des clusters

# Visualisation des clusters dans l’espace des deux premières dimensions :

# Utilisation de factoextra pour représenter les clusters
fviz_cluster(list(data = df_clust, cluster = df_style$cluster_kmodes), data = df_clust, ellipse.type = "euclid") +
  ggtitle("Visualisation des clusters avec DBScan")

# Graphique par variables

# Variables binaires
g_var_bin <- function(df, nom_var) {
  # Distribution des clusters par var
  g <- ggplot(df, aes(x = .data[[nom_var]], fill = cluster_kmodes)) +
    geom_bar(position = "dodge") +
    ggtitle(paste("Distribution des clusters pour", nom_var)) +
    theme_minimal()
  return(g)
}
g_var_bin(df_style, "app_swag_Classique")
g_var_bin(df_style, "app_swag_Casual")
g_var_bin(df_style, "app_swag_Sport")
g_var_bin(df_style, "app_swag_Other")
g_var_bin(df_style, "app_withTattoo")
g_var_bin(df_style, "animal_cat")
g_var_bin(df_style, "animal_dog")
g_var_bin(df_style, "animal_other")
g_var_bin(df_style, "animal_noPet")

### Clustering avec DBSCAN

Il semble que ça n'ait pas de sens étant donné la nature binaire des données

In [ ]:
library(dbscan)

# Pour touvre la valeur pour eps : Trouver la distance des 4 plus proches voisins
#
# Dans le graphique abline : 
#   Ajuster h pour estimer `eps`
#   eps correspond à h qui correspond à l'"épaule"
#   le h sert à tracer ne ligne rouge vis-à-vis l'épaule
#
# k devrait être égal à nb_var

#df_scaled <- scale(df_clust)
#head(df_scaled)

kNNdistplot(df_clust, k = 9)
h <- 1
abline(h = h, col = "red", lty = 2)

In [ ]:
# Appliquer DBSCAN
dbscan_result <- dbscan(df_clust, eps = h, minPts = 30)

# Afficher les clusters
print(dbscan_result)

dbscan_result$metric
dbscan_result$cluster


In [135]:
# Ajouter les résultats des clusters aux données
df_style$cluster_dbscan <- as.factor(dbscan_result$cluster)

In [ ]:
# Visualisation des clusters

# Visualisation des clusters dans l’espace des deux premières dimensions :

# Utilisation de factoextra pour représenter les clusters
fviz_cluster(dbscan_result, data = df_clust, ellipse.type = "euclid") +
  ggtitle("Visualisation des clusters avec DBScan")

# Graphique par variables

# Variables binaires
g_var_bin <- function(df, nom_var) {
  # Distribution des clusters par var
  g <- ggplot(df, aes(x = .data[[nom_var]], fill = cluster_dbscan)) +
    geom_bar(position = "dodge") +
    ggtitle(paste("Distribution des clusters pour", nom_var)) +
    theme_minimal()
  return(g)
}
g_var_bin(df_style, "app_swag_Classique")
g_var_bin(df_style, "app_swag_Casual")
g_var_bin(df_style, "app_swag_Sport")
g_var_bin(df_style, "app_swag_Other")
g_var_bin(df_style, "app_withTattoo")
g_var_bin(df_style, "animal_cat")
g_var_bin(df_style, "animal_dog")
g_var_bin(df_style, "animal_other")
g_var_bin(df_style, "animal_noPet")

### Reste du code

In [ ]:
library(cluster)
dissimilarity_matrix <- daisy(df_scaled, metric = "gower")

In [ ]:
dissimilarity_matrix

In [ ]:
# Création d'un tableau des fréquences
combinations <- as.data.frame(table(df_scaled))

# Filtrer uniquement les combinaisons ayant au moins une occurrence
combinations <- combinations[combinations$Freq > 0, ]

print(combinations)

library(ggplot2)

# Créer un identifiant pour chaque combinaison
combinations$combination <- apply(combinations[, 1:9], 1, paste, collapse = "")

# Visualiser les fréquences
ggplot(combinations, aes(x = reorder(combination, -Freq), y = Freq)) +
  geom_bar(stat = "identity") +
  labs(x = "Combinaisons", y = "Fréquence") +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))

In [ ]:
# Appliquer DBSCAN
dbscan_result <- dbscan(df_scaled, eps = 1, minPts = 30)

# Afficher les clusters
print(dbscan_result)

# Visualisation
# library(ggplot2)
# df_scaled$cluster <- as.factor(dbscan_result$cluster)
# ggplot(df_scaled, aes(x = mpg, y = disp, color = cluster)) +
#   geom_point(size = 3) +
#   labs(title = "DBSCAN Clustering", color = "Cluster")

In [16]:
# Appliquer PCA pour réduire la dimensionnalité
pca_result <- prcomp(df_scaled, scale = TRUE)

# Extraire les 2 premières composantes principales pour le graphique
pca_data <- data.frame(pca_result$x[, 1:2])  # Première et deuxième composantes principales

# Ajouter les résultats de DBSCAN aux données PCA
pca_data$cluster <- as.factor(dbscan_result$cluster)

In [ ]:
library(ggplot2)

# Créer un graphique des clusters DBSCAN
ggplot(pca_data, aes(x = PC1, y = PC2, color = cluster)) +
  geom_point(size = 3) +
  labs(title = "DBSCAN Clustering avec PCA",
       x = "Composante Principale 1",
       y = "Composante Principale 2",
       color = "Cluster") +
  theme_minimal()

In [ ]:
library(Rtsne)

# Appliquer t-SNE pour réduire à 2 dimensions
tsne_result <- Rtsne(df_scaled, dims = 2)

# Créer un dataframe avec les résultats t-SNE
tsne_data <- data.frame(tsne_result$Y)
tsne_data$cluster <- as.factor(dbscan_result$cluster)

# Visualisation t-SNE
ggplot(tsne_data, aes(x = V1, y = V2, color = cluster)) +
  geom_point(size = 3) +
  labs(title = "DBSCAN Clustering avec t-SNE",
       x = "t-SNE Dimension 1",
       y = "t-SNE Dimension 2",
       color = "Cluster") +
  theme_minimal()

In [ ]:
library(dplyr)
# Ajouter les résultats des clusters aux données
df_scaled$cluster <- as.factor(dbscan_result$cluster)
# Calculer la moyenne et l'écart-type pour chaque variable par cluster
summary_clusters <- df_scaled %>%
  group_by(cluster) %>%
  summarise(across(starts_with("app_") | starts_with("animal_"), list(mean = mean, sd = sd), .names = "{.col}_{.fn}"))

# Afficher le résumé des statistiques pour chaque cluster
View(summary_clusters)